In [1]:
from USTs import USTs
import datetime
from datetime import timedelta
from typing import List
from dateutil.relativedelta import relativedelta
from scipy.optimize import newton
import pandas_market_calendars as mcal
import pandas as pd
import numpy as np

In [2]:
# Bill pricing
issue_date = datetime.date(2025, 2, 4)
maturity_date = datetime.date(2025, 4, 24)

bill_1 = USTs(auction_data= None, price_data=None).get_bill_discount_rate(price=99.0808021,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_1
issue_date = datetime.date(2025, 2, 4)
maturity_date = datetime.date(2025, 4, 24)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=99.0808021,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2
issue_date = datetime.date(1990, 6, 7)
maturity_date = datetime.date(1991, 6, 6)
bill_2 = USTs(auction_data= None, price_data=None).get_bill_BEYTM(price=92.265000,
                                                                          issue_date=issue_date,
                                                                          maturity_date=maturity_date)
bill_2

8.237

In [3]:
holiday_array = mcal.get_calendar('NYSE').holidays().holidays
start, end = datetime.date(1990, 1, 1), datetime.date(2060, 1, 1)
holidays_list = [pd.to_datetime(np_date).date() for np_date in holiday_array]
holidays = [holiday for holiday in holidays_list if start < holiday < end]

In [4]:
nyse_calendar = mcal.get_calendar('NYSE').regular_holidays
def adjust_for_bad_day(date: datetime.date) -> datetime.date: # type: ignore
    while date.weekday() in [5, 6] or date in holidays:
        date = date + timedelta(days=1)
    return date

def get_coupon_dates(issue_date, maturity_date) -> List[datetime.date]: # type: ignore
    payment_set = []
    payment_set.append(maturity_date)
    current_date = maturity_date

    while current_date > issue_date:
        current_date = maturity_date - relativedelta(months=len(payment_set) * 6)
        current_date = adjust_for_bad_day(current_date)
        if current_date > issue_date:
            payment_set.append(current_date)
    payment_set.sort()
    return payment_set

def get_dates_and_cashflows(issue_date: datetime.date, maturity_date: datetime.date, as_of_date: datetime.date, coupon: float, get_days: bool = True, FV: int = 100):
    dates = get_coupon_dates(issue_date, maturity_date)
    coupon_amt = coupon / 2
    return_list = list()
    if get_days:
        days = [(date - as_of_date).days for date in dates]
        for day in days[:-1]:
            return_list.append((day, coupon_amt))
        return_list.append((days[-1], coupon_amt + FV))
    else:
        for date in dates[:-1]:
            return_list.append((date, coupon_amt))
        return_list.append((dates[-1], coupon_amt + FV))
    return return_list

# Calculating bond price
def get_next_coupon_days(days_and_cashflows) -> int:
    for days, cashflow in days_and_cashflows:
        if days > 0:
            day = days
            return day

def get_last_coupon_days(days_and_cashflows, issue_date) -> int:
    current_day = -100000
    for day, cashflows in days_and_cashflows:
        if day < 0:
            if day > current_day:
                current_day = day
    if current_day == -100000:
        current_day = (issue_date - datetime.datetime.now().date()).days
    return current_day

def get_accrued(days_and_cashflows, coupon, issue_date) -> float:
    next_payment = get_next_coupon_days(days_and_cashflows=days_and_cashflows)
    last_payment = get_last_coupon_days(days_and_cashflows=days_and_cashflows, issue_date=issue_date)
    accrued = coupon / 2 * abs(last_payment) / (next_payment - last_payment)
    return accrued

def calculate_bond_price(issue_date: datetime.date,
                         maturity_date: datetime.date,
                         as_of_date: datetime.date,
                         coupon: float,
                         discount_rate: float,
                         dirty: bool = False) -> float:
    
    days_and_cashflows = get_dates_and_cashflows(issue_date=issue_date,
                                                 maturity_date=maturity_date,
                                                 as_of_date=as_of_date,
                                                 coupon=coupon,
                                                 get_days=True,
                                                 FV=100)
    # Dirty price
    PV = 0
    DISCOUNT_RATE = discount_rate / 100
    for day, cashflow in days_and_cashflows:
        if day > 0:
            pmt_value = cashflow/((1 + DISCOUNT_RATE)**(day / (365.25 / 2)))
            PV += pmt_value
    
    if not dirty:
        accrued = get_accrued(days_and_cashflows=days_and_cashflows, coupon=coupon, issue_date=issue_date)
        print(f"Accrued: {accrued}")
        PV -= accrued
    return PV
def error_function(discount_rate: float,
                   price: float,
                   issue_date: datetime.date,
                   maturity_date: datetime.date,
                   as_of_date: datetime.date,
                   coupon: float,
                   dirty: bool=False):
    calculated_price = calculate_bond_price(issue_date=issue_date,
                                            maturity_date=maturity_date,
                                            discount_rate=discount_rate,
                                            coupon=coupon,
                                            as_of_date=as_of_date,
                                            dirty=dirty)
    error = price - calculated_price
    return error


def get_coupon_ytm(price: float, 
            issue_date: datetime.date,
            maturity_date: datetime.date,
            as_of_date: datetime.date,
            coupon: float,
            dirty: bool=False):
    guess = coupon / 2 / 100

    try:
        ytm = newton(
            func=error_function,
            x0=guess,
            args=(price, issue_date, maturity_date, as_of_date, coupon, dirty)
        )
        return round(float(ytm * 2), 6)
    except RuntimeError:
        return None

In [5]:
PRICE = 94
ISSUE = datetime.datetime(2024, 11, 15)
MATURITY = datetime.datetime(2054, 11, 15)
RATE = 4.500
SETTLEMENT = datetime.datetime(2025, 6, 23)

In [6]:
get_coupon_ytm(price=PRICE,
        issue_date=ISSUE.date(),
        maturity_date=MATURITY.date(),
        as_of_date=SETTLEMENT.date(),
        coupon=RATE)

Accrued: 0.4717741935483871
Accrued: 0.4717741935483871
Accrued: 0.4717741935483871
Accrued: 0.4717741935483871
Accrued: 0.4717741935483871
Accrued: 0.4717741935483871
Accrued: 0.4717741935483871
Accrued: 0.4717741935483871
Accrued: 0.4717741935483871


4.88616

In [7]:
import rateslib as rl
bond = rl.FixedRateBond(
    effective=ISSUE,
    termination=MATURITY,
    fixed_rate=RATE,
    spec="us_gb"  # US Government Bond
)
bond.ytm(price=PRICE, settlement=SETTLEMENT, dirty=False)

4.886469641442125

In [8]:
bond.accrued(settlement=SETTLEMENT)

0.47690217391304346

In [9]:
start = datetime.date(2025, 5, 15)
end = start + relativedelta(months=6)
end-start # 184
SETTLEMENT.date() - start # 39

4.5 / 2 * 39 / 184

0.47690217391304346

In [10]:
USTs(auction_data=None,price_data=None).get_coupon_ytm(price=PRICE, issue_date=ISSUE.date(),
                    maturity_date=MATURITY.date(),
                    as_of_date=SETTLEMENT.date(),
                    coupon=RATE)

4.88616